In [32]:
import os.path

import numpy as np
import glob

from vis.deprecated_vis import *

from data.PATHS import DATA_PATH

data_path = f'{DATA_PATH}/valeo_emre/'
npz_file = glob.glob(f'{data_path}/*.npz')[3]
data_file = np.load(npz_file, allow_pickle=True)

odom_list = data_file['odom_list']
scan_list = data_file['scan_list']



In [40]:
# processing
from tqdm import tqdm
data_npz = {}
os.makedirs(npz_file[:-4], exist_ok=True)

pcl_list = []
global_list = []
for i in tqdm(range(180, len(scan_list) - 1)):
    
    pc1 = np.stack((scan_list[i][0]['x'],
        scan_list[i][0]['y'],
        scan_list[i][0]['z'],
        scan_list[i][0]['i'],
        scan_list[i][0]['layer_id'])).T
    
    pc2 = np.stack((scan_list[i+1][0]['x'],
        scan_list[i+1][0]['y'],
        scan_list[i+1][0]['z'],
        scan_list[i+1][0]['i'],
        scan_list[i+1][0]['layer_id'])).T  
    
    lidar1 = scan_list[i][0]['lidar'] == 0
    lidar2 = scan_list[i+1][0]['lidar'] == 0
    
    pose1 = odom_list['transformation'][i]
    pose2 = odom_list['transformation'][i+1]
    
    
    
    gt_flow = - np.ones((pc1.shape[0], 3))
    data_npz = {
        'pc1': pc1[lidar1,:3],
        'pc2': pc2[lidar2,:3],
        'flow' : gt_flow,
        'pose1': pose1,
        'pose2': pose2
    }
    
    pcl_list.append(np.insert(pc1.copy()[lidar1], 3, i, axis=1))
    global_list.append((np.insert(pc1.copy()[lidar1], 3, 1, axis=1)[:,:4] @ pose1.T)[:,:3])
    np.savez(f'{data_path}/{os.path.basename(npz_file)[:-4]}/{i:06d}.npz', **data_npz)
    
    print(pose1)
    
    if len(pcl_list) == 3: break


    # scan_list[0][0]['lidar']

# visualize_points3D(pc1, pc1[:,3] == 0, lookat=[0,0,0])
pc = np.concatenate(pcl_list)
gl = np.concatenate(global_list)

# visualize_points3D(pc, pc[:,5], lookat=[0,0,0])
# visualize_points3D(gl, gl[:,2], lookat=[0,0,0])


  3%|▎         | 2/73 [00:00<00:02, 32.69it/s]

[[ 1.73154550e-02  9.99847487e-01 -2.27558769e-03  2.09577000e+01]
 [-9.99849706e-01  1.73134441e-02 -9.00436482e-04 -7.26474000e+00]
 [-8.60900894e-04  2.29083715e-03  9.99997005e-01 -9.35357000e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 1.65055695e-02  9.99861159e-01 -2.28647076e-03  2.09600000e+01]
 [-9.99863306e-01  1.65033067e-02 -1.00503472e-03 -7.30740000e+00]
 [-9.67160849e-04  2.30274689e-03  9.99996881e-01 -9.17382000e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 1.60356313e-02  9.99869052e-01 -2.17665248e-03  2.09604000e+01]
 [-9.99870914e-01  1.60333919e-02 -1.04240545e-03 -7.33923000e+00]
 [-1.00736983e-03  2.19308713e-03  9.99997088e-01 -9.10506000e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [35]:
# pc to voxel grid
voxel_size = 0.2
max_x = 35
max_y = 35
max_z = 3
min_x = -35
min_y = -35
min_z = -3


voxel_grid = np.zeros((int((max_x - min_x) / voxel_size) + 2, int((max_y - min_y) / voxel_size) + 2, int((max_z - min_z) / voxel_size) + 2))
all_time_voxel = voxel_grid.copy()

for t in np.unique(pc[:,3]):
    pc_in_range = pc[(pc[:,0] > min_x) & (pc[:,0] < max_x) & (pc[:,1] > min_y) & (pc[:,1] < max_y) & (pc[:,2] > min_z) & (pc[:,2] < max_z) & (pc[:,3] == t)]
    pc_xyz = (pc_in_range[:,:3] - np.array([min_x, min_y, min_z])) / voxel_size
    
    voxel_grid = np.zeros((int((max_x - min_x) / voxel_size) + 2, int((max_y - min_y) / voxel_size) + 2, int((max_z - min_z) / voxel_size) + 2))
    voxel_grid[pc_xyz[:,0].astype(int), pc_xyz[:,1].astype(int), pc_xyz[:,2].astype(int)] = 1
    all_time_voxel += voxel_grid

# color
all_time_voxel = all_time_voxel.astype(int)
pc_in_range = pc[(pc[:,0] > min_x) & (pc[:,0] < max_x) & (pc[:,1] > min_y) & (pc[:,1] < max_y) & (pc[:,2] > min_z) & (pc[:,2] < max_z)]
pc_xyz = (pc_in_range[:,:3] - np.array([min_x, min_y, min_z])) / voxel_size
accum_time_feature = all_time_voxel[pc_xyz[:,0].astype(int), pc_xyz[:,1].astype(int), pc_xyz[:,2].astype(int)]

visualize_points3D(pc_in_range, accum_time_feature, lookat=[0,0,0])
visualize_points3D(pc_in_range, pc_in_range[:, 3], lookat=[0,0,0])


In [43]:
import numpy as np
import glob
import os 
import torch
from vis.deprecated_vis import *
# argoverse
def find_robust_weighted_rigid_alignment(A, B, weights, use_epsilon_on_weights=False):
    """
    Calculates the weighted rigid transformation that aligns two sets of points.
    Args:
        A (torch.Tensor): A tensor of shape (batch_size, num_points, 3) containing the first set of points.
        B (torch.Tensor): A tensor of shape (batch_size, num_points, 3) containing the second set of points.
        weights (torch.Tensor): A tensor of shaep (batch_size, num_points) containing weights.
        use_epsilon_on_weights (bool): A condition if to use eps for weights.
    Returns:
        torch.Tensor: A tensor of shape (batch_size, 4, 4) containing the rigid transformation matrix that aligns A to B.
    """
    # assert (weights >= 0.0).all(), "Negative weights found"
    # if use_epsilon_on_weights:
    #     weights += torch.finfo(weights.dtype).eps
    #     count_nonzero_weighted_points = torch.sum(weights > 0.0, dim=-1)
    #     not_enough_points = count_nonzero_weighted_points < 3
    # else:
    #     # Add eps if not enough points with weight over zero
    #     count_nonzero_weighted_points = torch.sum(weights > 0.0, dim=-1)
    #     not_enough_points = count_nonzero_weighted_points < 3
    #     eps = not_enough_points.float() * torch.finfo(weights.dtype).eps
    #     weights += eps.unsqueeze(-1)
    # assert not not_enough_points, f"pcl0 shape {A.shape}, pcl1 shape {B.shape}, points {count_nonzero_weighted_points}"

    weights = weights.unsqueeze(-1)
    sum_weights = torch.sum(weights, dim=1)

    A_weighted = A * weights
    B_weighted = B * weights

    a_mean = A_weighted.sum(axis=1) / sum_weights.unsqueeze(-1)
    b_mean = B_weighted.sum(axis=1) / sum_weights.unsqueeze(-1)

    A_c = A - a_mean
    B_c = B - b_mean
    # Covariance matrix
    H = ((A_c * weights).transpose(1, 2) @ B_c) / sum_weights
    U, S, V = torch.svd(H)
    # Rotation matrix
    R = V @ U.transpose(1, 2)
    # Translation vector
    t = b_mean.transpose(1, 2) - (R @ a_mean.transpose(1, 2))

    T = torch.cat((R, t), dim=2)
    T = torch.cat((T, torch.tensor([[[0,0,0,1]]], device=A.device)), dim=1)
    return T

def transform_pc(pts, pose):

    '''

    :param pts: point cloud
    :param pose: 4x4 transformation matrix
    :return:
    '''
    transformed_pts = np.insert(pts.copy(), 3, 1, axis=1)
    transformed_pts[:, 3] = 1
    transformed_pts[:, :3] = (transformed_pts[:, :4] @ pose.T)[:, :3]

    return transformed_pts


paths = sorted(glob.glob(f'{os.path.expanduser("~")}/data/sceneflow/argoverse/val/f9*/*.npz'))[0]
# paths = sorted(glob.glob(f'{os.path.expanduser("~")}/data/sceneflow/kittisf/all_data_format/*.npz'))[0]
pc_list = []
data = np.load(paths, allow_pickle=True)
pc1 = torch.from_numpy(data['pc1'])[None, :]
pc2 = torch.from_numpy(data['pc2'])[None, :]
flow = torch.from_numpy(data['flow'])[None, :]

background_weights = torch.ones(flow.shape[:2])
# background_weights = torch.from_numpy(data['inst_pc1'] == 0).to(torch.float)[None, :]

pose = find_robust_weighted_rigid_alignment(pc1, pc1 + flow, weights=background_weights)

global_pc = transform_pc(pc1[0].clone().numpy(), pose[0].numpy())
# global_pc = pc1[0].numpy() - pose[0].numpy()[:3, 3]

visualize_multiple_pcls(*[pc1[0].numpy(), global_pc[:,:3], pc2[0].numpy()], lookat=[0,0,0])
# visualize_multiple_pcls(*[pc1[0].numpy() + flow[0].numpy(), pc2[0].numpy()], lookat=[0,0,0])


In [45]:
import numpy as np
ids = np.arange(10)
probs = np.zeros((ids.shape[0], ids.max()))
for i in range(ids.max()):
    probs[ids == i, i] = 1
print(probs)


[[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]
